# Mining Processes From Smart Contracts

In [1]:
from web3 import Web3
w3 = Web3(Web3.WebsocketProvider("ws://127.0.0.1:8545", websocket_kwargs={'timeout': 60}))
w3.isConnected()

True

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

In [32]:
import sha3
k = sha3.keccak_256()
k.update(b"sendRequest(uint8,bytes)")
pp.pprint(k.hexdigest())
task_1 = k.hexdigest()[:8]
print(task_1)


from Crypto.Hash import keccak
keccak_hash = keccak.new(digest_bits=256)
keccak_hash.update(b"sendResponse(uint8,bytes)")
pp.pprint(keccak_hash.hexdigest())
task_2 = keccak_hash.hexdigest()[:8]
print(task_2)

'281b37f71b3ebd930fcc3871d8464e7c95d61f99625c5b629f5847d345ae81bc'
281b37f7
'8843f5670ed573903f48e42de93613234934b6b389553e6622b288619a7f5fc5'
8843f567


In [3]:
CONTRACT_ADDRESS_A = "0x54497Da55F80B125a2050373fb7230e5A4eFA077"
CONTRACT_ADDRESS_B = "0x7F79072c37A02e1F9474aa18Be0B4FeAA68cE624"
PARTICIPANTS_CONTAINER = "0xcd23eCBBdBcC03aE1Bcf658Fd3893F2c6f87DeEf"


block = w3.eth.getCode(CONTRACT_ADDRESS_A)
contract = w3.eth.contract(CONTRACT_ADDRESS_A)

#print(block)
#print(contract)

In [6]:
contract_a_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": CONTRACT_ADDRESS_A
})

contract_a_eventlist = contract_a_filter.get_all_entries()
pp.pprint(contract_a_eventlist)

[ { 'address': '0x54497Da55F80B125a2050373fb7230e5A4eFA077',
    'blockHash': HexBytes('0x80f6b0d77e130682724d30530d194494b90112dca8416335f3ce52ba6e2371eb'),
    'blockNumber': 15,
    'data': '0x000000000000000000000000483b7ebdd347a754dafe63c2681302f8a465bef000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000000',
    'logIndex': 2,
    'topics': [ HexBytes('0x483bc8f670acc41b1a377626890eb8f1c05eab1ca1310cf4c2b4ffd48ccd7ce9')],
    'transactionHash': HexBytes('0xfc018bba069101a296c6366b0610b05bfafb1b788279a0e4c83f858a8d42c39c'),
    'transactionIndex': 0,
    'type': 'mined'},
  { 'address': '0x54497Da55F80B125a2050373fb7230e5A4eFA077',
    'blockHash': HexBytes('0xb96d9aa0dc895ed039e7350c916573a6875c1149f419e312ab883768006d3372'),
    'blockNumber': 25,
    'data': '0x000000000000000000000000b1b83a155f16d292a0a64ae2bea543f1fb9f9b31000000000000000000000000000000000000000000000000000000000000004000000000000000000

In [ ]:
contract_b_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": CONTRACT_ADDRESS_B
})

contract_b_eventlist = contract_b_filter.get_all_entries()
pp.pprint(contract_b_eventlist)

In [ ]:
participants_container_filter = w3.eth.filter({
    "fromBlock": "0x0",
    "toBlock": "latest",
    "address": PARTICIPANTS_CONTAINER
})

participants_container_eventlist = participants_container_filter.get_all_entries()
pp.pprint(participants_container_eventlist)

In [ ]:
### From BACHOFNER ET AL.
### REWRITE BEFORE COMMITTING

from eth_utils import decode_hex, encode_hex
from rlp import encode

def get_all_transactions_for_contract_address(contract_address=CONTRACT_ADDRESS_A, start_block_number=0, end_block_number=40000):
    contract_address_transactions = list()
    pp.pprint(f"Searching for transactions to/from account {contract_address} within blocks {start_block_number} and {end_block_number}")
    
    for block_number in range(start_block_number, end_block_number+1):
        block = w3.eth.getBlock(block_number, full_transactions=True)
        
        transactions = block["transactions"]
        for transaction in transactions:
            if transaction["from"] == contract_address:
                contract_address_transactions.append(transaction)
                continue
          
            if "to" in transaction and transaction["to"] == contract_address:
                contract_address_transactions.append(transaction)
          
    return contract_address_transactions


def get_all_contract_transactions(start_block_number=0, end_block_number=40000):
    contract_address_transactions_receipts = list()
    contract_address_transactions = list()
    pp.pprint(f"Searching for all contract transactions within blocks {start_block_number} and {end_block_number}")
    
    for block_number in range(start_block_number, end_block_number+1):
        block = w3.eth.getBlock(block_number, full_transactions=True)
        
        transactions = block["transactions"]
        for transaction in transactions:
            trans_rec = w3.eth.getTransactionReceipt(transaction["hash"])
            
            if "contractAddress" in trans_rec and trans_rec["contractAddress"] is not None:
                contract_address_transactions_receipts.append(trans_rec)
                contract_address_transactions.append(transaction)
          
    return contract_address_transactions, contract_address_transactions_receipts

def get_contract_transactions_for_contract_address(contract_address=CONTRACT_ADDRESS_A, start_block_number=0, end_block_number=40000):
    contract_address_transactions_receipts = list()
    contract_address_transactions = list()
    pp.pprint(f"Searching for contract transactions to/from account {contract_address} within blocks {start_block_number} and {end_block_number}")
    
    for block_number in range(start_block_number, end_block_number+1):
        block = w3.eth.getBlock(block_number, full_transactions=True)
        
        transactions = block["transactions"]
        for transaction in transactions:
            trans_rec = w3.eth.getTransactionReceipt(transaction["hash"])
            
            if transaction["to"] == contract_address:
                contract_address_transactions.append(transaction)
                contract_address_transactions_receipts.append(trans_rec)
            if "contractAddress" in trans_rec and trans_rec["contractAddress"] == contract_address:
                contract_address_transactions_receipts.append(trans_rec)
                contract_address_transactions.append(transaction)
          
    return contract_address_transactions, contract_address_transactions_receipts




number = w3.eth.getBlock('latest').number


In [ ]:
# get transaction receipts where contract was involved
trans, recs = get_contract_transactions_for_contract_address(CONTRACT_ADDRESS_A, 0, number)
print("\n Transactions: \n")
pp.pprint(trans) 
print("\n Receipts: \n")
pp.pprint(recs)

In [17]:
# get all transactions and recipts of all contract transactions
all_trans, all_recs = get_all_contract_transactions(0, number)
print("\n Transactions: \n")
pp.pprint(all_trans)
print("\n Receipts: \n")
pp.pprint(all_recs)

'Searching for all contract transactions within blocks 0 and 40703'

 Transactions: 

[ AttributeDict({'hash': HexBytes('0xe754d6e87ca47f06f5ec022e6cdf0697a48e208e53c13473bc31d5291b8eb987'), 'nonce': 0, 'blockHash': HexBytes('0x596eaab3b73c89b7c174fe404ed3431c53dc6cf9eeaa10637b19c8253a7c5094'), 'blockNumber': 10, 'transactionIndex': 0, 'from': '0xBF8e90753D5a55ffC3683AB3bf8b7C82Ce52113d', 'to': None, 'value': 0, 'gas': 3000000, 'gasPrice': 0, 'input': '0x608060405234801561001057600080fd5b506117e3806100206000396000f3006080604052600436106100405763ffffffff7c0100000000000000000000000000000000000000000000000000000000600035041663cf5ba53f8114610045575b600080fd5b34801561005157600080fd5b50610065600480356024810191013561008e565b6040805173ffffffffffffffffffffffffffffffffffffffff9092168252519081900360200190f35b600033838361009b610181565b73ffffffffffffffffffffffffffffffffffffffff84168152604060208201818152908201839052606082018484808284378201915050945050505050604051809103906000f0801580156100f4573d60008

In [ ]:
# get transactions for one transactionHash from deploying transaction (to: none), from block belonging to event
#transacts = get_all_transactions_for_contract_address("0x54497Da55F80B125a2050373fb7230e5A4eFA077", 0, number)
#print("\n")
#pp.pprint(transacts)

In [42]:
def normalise(x, allow_blank=False):
    if allow_blank and x == '':
        return ''
    if len(x) in (42, 50) and x[:2] == '0x':
        x = x[2:]
    if len(x) in (40, 48):
        x = decode_hex(x)
    if len(x) == 24:
        assert len(x) == 24 and ssha3.keccak_256(x[:20])[:4] == x[-4:]
        x = x[:20]
    if len(x) != 20:
        raise Exception("Invalid address format: %r" % x)
    return x

def mk_contract_address(sender, nonce):
    normie_sender = normalise(sender)
    res = sha3.keccak_256(encode([normie_sender, nonce])).hexdigest()
    return res[12:]

# from or contract address of previous contract transactions
#contract_address = "0x" + mk_contract_address("0xBF8e90753D5a55ffC3683AB3bf8b7C82Ce52113d", 14)
contract_address = "0x" + mk_contract_address(CONTRACT_ADDRESS_A, 5)
get_all_transactions_for_contract_address(contract_address, 0, number)

('Searching for transactions to/from account '
 '0x680fd0dd43f37dfa57cf826438f970edb6affa9a24a06013bdd5 within blocks 0 and '
 '40703')


[]

In [38]:
# get transaction receipts where contract was involved
trans, recs = get_contract_transactions_for_contract_address("0xE1EfC7F8F50e2607Ae7c03E27c283339A95373f0", 0, number)
print("\n Transactions: \n")
pp.pprint(trans) 
print("\n Receipts: \n")
pp.pprint(recs)

('Searching for contract transactions to/from account '
 '0xE1EfC7F8F50e2607Ae7c03E27c283339A95373f0 within blocks 0 and 40703')

 Transactions: 

[ AttributeDict({'hash': HexBytes('0xadc3736d1f3c7d9339d98aa030a817cb1af7a4c5ce04fa8f58b19da5f4a7a7ed'), 'nonce': 151, 'blockHash': HexBytes('0x7b679171abd13ec5164ea691aca59ae0c3c68a6a8584d4ea582d8551ea626b05'), 'blockNumber': 864, 'transactionIndex': 0, 'from': '0xBF8e90753D5a55ffC3683AB3bf8b7C82Ce52113d', 'to': None, 'value': 0, 'gas': 3000000, 'gasPrice': 0, 'input': '0x60806040526001805460ff1916905534801561001a57600080fd5b5060405161154838038061154883398101604081815282516020808501516000805491909601600281015164010000000061ffff909116027fffffffffff0000000000000000000000000000000000000000ffffffffffffff909216670100000000000000600160a060020a038616021765ffff0000000019169190911786558185528051828601528051929590947f3b4a808d54c495ced21b3f493818c8c5a90f41e600c8b12208571402dcd23cec9486949193849391840192908601918190849084905b838110156100f85781810151838